Human Activity Recognition

Human Activity Recognition is the problem of predicting what a person is doing based on a trace of their movement using sensors. The "Human Activity Detection #Dataset" includes data collected from 34 subjects, each of whom were asked to perform 18 tasks for 3 minutes each. Each subject had a smartwatch placed on his/her dominant hand and a smartphone in their pocket. The data collection was controlled by a custom-made app that ran on the #smartphone and #smartwatch. The sensor data that was collected was from the accelerometer and gyroscope on both the smartphone and smartwatch, yielding four total sensors. The sensor data was collected at a rate of 20 Hz (i.e., every 50ms). The smartphone was either the Google Nexus 5/5X or Samsung Galaxy S5 running Android 6.0 (Marshmallow). The smartwatch was the LG G Watch running Android Wear 1.5.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1624_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1605_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1642_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1637_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1600_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1649_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/data_1602_accel_phone.txt
/kaggle/input/smartphone-and-smartwatch-activity-and-biometric

In [3]:
from __future__ import print_function
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
#import coremltools
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


Phone Accelerometer Train/test

In [4]:
columns=['user','activity','time','x','y','z']

data_phone_accel_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel'):
    for filename in filenames:
        df = pd.read_csv('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/accel/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_phone_accel_sum=pd.concat([data_phone_accel_sum,temp])

In [5]:
data_phone_accel_sum['z'] = data_phone_accel_sum['z'].str.replace(';','')
data_phone_accel_sum['activity'].value_counts()
data_phone_accel_sum['x']=data_phone_accel_sum['x'].astype('float')
data_phone_accel_sum['y']=data_phone_accel_sum['y'].astype('float')
data_phone_accel_sum['z']=data_phone_accel_sum['z'].astype('float')
data_phone_accel_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4804403 entries, 0 to 64307
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 256.6+ MB


Phone Gyro files import Train/test

In [6]:

data_phone_gyro_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/gyro'):
    for filename in filenames:
        df = pd.read_csv('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/phone/gyro/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_phone_gyro_sum=pd.concat([data_phone_gyro_sum,temp])

In [7]:
data_phone_gyro_sum['z'] = data_phone_gyro_sum['z'].str.replace(';','')

data_phone_gyro_sum['x']=data_phone_gyro_sum['x'].astype('float')
data_phone_gyro_sum['y']=data_phone_gyro_sum['y'].astype('float')
data_phone_gyro_sum['z']=data_phone_gyro_sum['z'].astype('float')

data_phone_gyro_sum['activity'].value_counts()
data_phone_gyro_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3608635 entries, 0 to 64243
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 192.7+ MB


Watch Gyro files import train/test

In [8]:

data_watch_gyro_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/watch/gyro'):
    for filename in filenames:
        df = pd.read_csv('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/watch/gyro/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_watch_gyro_sum=pd.concat([data_watch_gyro_sum,temp])

In [9]:
data_watch_gyro_sum['z'] = data_watch_gyro_sum['z'].str.replace(';','')
data_watch_gyro_sum['x']=data_watch_gyro_sum['x'].astype('float')
data_watch_gyro_sum['y']=data_watch_gyro_sum['y'].astype('float')
data_watch_gyro_sum['z']=data_watch_gyro_sum['z'].astype('float')

data_watch_gyro_sum['activity'].value_counts()
data_watch_gyro_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440342 entries, 0 to 64857
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 183.7+ MB


Watch accelorometer files import train test

In [10]:

data_watch_accel_sum = pd.DataFrame(data=None,columns=columns)
for dirname, _, filenames in os.walk('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/watch/accel'):
    for filename in filenames:
        df = pd.read_csv('/kaggle/input/smartphone-and-smartwatch-activity-and-biometrics/wisdm-dataset/wisdm-dataset/raw/watch/accel/'+filename , sep=",", header=None)
        temp=pd.DataFrame(data=df.values, columns=columns)
        data_watch_accel_sum=pd.concat([data_watch_accel_sum,temp])

In [11]:
data_watch_accel_sum['z'] = data_watch_accel_sum['z'].str.replace(';','')
data_watch_accel_sum['x']=data_watch_accel_sum['x'].astype('float')
data_watch_accel_sum['y']=data_watch_accel_sum['y'].astype('float')
data_watch_accel_sum['z']=data_watch_accel_sum['z'].astype('float')

data_watch_accel_sum['activity'].value_counts()
data_watch_accel_sum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3777046 entries, 0 to 64904
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 201.7+ MB


Combining Phone accel and gyro data

In [12]:
df_phone = pd.DataFrame(data=None, columns=columns)
df_phone['user']= data_phone_accel_sum['user'].head(3608635)
df_phone['activity']= data_phone_accel_sum['activity'].head(3608635)
df_phone['time']= data_phone_accel_sum['time'].head(3608635)
df_phone['x'] = data_phone_gyro_sum['x'].values + data_phone_accel_sum['x'].head(3608635).values
df_phone['y'] = data_phone_gyro_sum['y'].values + data_phone_accel_sum['y'].head(3608635).values
df_phone['z'] = data_phone_gyro_sum['z'].values + data_phone_accel_sum['z'].head(3608635).values

Combining watch acccel and gyro data

In [13]:
df_watch = pd.DataFrame(data=None, columns=columns)
df_watch['user']= data_watch_accel_sum['user'].head(3440342)
df_watch['activity']= data_watch_accel_sum['activity'].head(3440342)
df_watch['time']= data_watch_accel_sum['time'].head(3440342)
df_watch['x'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['x'].head(3440342).values
df_watch['y'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['y'].head(3440342).values
df_watch['z'] = data_watch_gyro_sum['x'].values + data_watch_accel_sum['z'].head(3440342).values

In [14]:
df_phone['activity'].value_counts()

M    215128
O    212249
A    210850
H    207248
G    206081
E    206059
K    205276
P    202779
D    200784
B    199484
L    198967
I    196133
R    195604
Q    193432
S    192786
J    190860
F    188205
C    186710
Name: activity, dtype: int64

In [15]:
df_watch['activity'].value_counts()

K    197861
Q    197333
D    195001
P    194777
E    193017
O    192089
I    192031
M    191473
H    191451
A    190710
G    190702
S    189712
C    189294
R    189241
B    187769
F    187119
L    185667
J    185095
Name: activity, dtype: int64

Combining Phone and Watch Data

In [16]:
df_phone_watch = pd.DataFrame(data=None, columns=columns)
df_phone_watch['user']= df_phone['user'].head(3440342)
df_phone_watch['activity']= df_phone['activity'].head(3440342)
df_phone_watch['time']= df_phone['time'].head(3440342)
df_phone_watch['x'] = df_watch['x'].values + df_phone['x'].head(3440342).values
df_phone_watch['y'] = df_watch['y'].values + df_phone['y'].head(3440342).values
df_phone_watch['z'] = df_watch['z'].values + df_phone['z'].head(3440342).values

In [17]:
df_phone_watch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440342 entries, 0 to 45498
Data columns (total 6 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user      object 
 1   activity  object 
 2   time      object 
 3   x         float64
 4   y         float64
 5   z         float64
dtypes: float64(3), object(3)
memory usage: 183.7+ MB


In [18]:
df_phone_watch['activity'].value_counts()

A    201918
H    198317
M    197197
G    197150
E    197128
P    196413
O    194325
K    192772
R    192031
D    191851
B    190552
Q    189861
S    189213
I    187202
L    186464
J    180895
F    179275
C    177778
Name: activity, dtype: int64

In [19]:
Fs = 20

In [20]:
activities = df_phone_watch['activity'].value_counts().index

In [21]:
df_phone_watch = df_phone_watch.drop(['user', 'time'], axis=1)

In [22]:
df_phone_watch['activity'].value_counts()

A    201918
H    198317
M    197197
G    197150
E    197128
P    196413
O    194325
K    192772
R    192031
D    191851
B    190552
Q    189861
S    189213
I    187202
L    186464
J    180895
F    179275
C    177778
Name: activity, dtype: int64

In [23]:
df_a = df_phone_watch[df_phone_watch['activity']=='A'].head(174604)
df_m = df_phone_watch[df_phone_watch['activity']=='M'].head(174604)
df_k = df_phone_watch[df_phone_watch['activity']=='K'].head(174604)
df_p = df_phone_watch[df_phone_watch['activity']=='P'].head(174604)
df_e = df_phone_watch[df_phone_watch['activity']=='E'].head(174604)
df_o = df_phone_watch[df_phone_watch['activity']=='O'].head(174604)
df_c = df_phone_watch[df_phone_watch['activity']=='C'].head(174604)
df_d = df_phone_watch[df_phone_watch['activity']=='D'].head(174604)
df_l = df_phone_watch[df_phone_watch['activity']=='L'].head(174604)
df_b = df_phone_watch[df_phone_watch['activity']=='B'].head(174604)
df_h = df_phone_watch[df_phone_watch['activity']=='H'].head(174604)
df_f = df_phone_watch[df_phone_watch['activity']=='F'].head(174604)
df_g = df_phone_watch[df_phone_watch['activity']=='G'].head(174604)
df_q = df_phone_watch[df_phone_watch['activity']=='Q'].head(174604)
df_r = df_phone_watch[df_phone_watch['activity']=='R'].head(174604)
df_s = df_phone_watch[df_phone_watch['activity']=='S'].head(174604)
df_i = df_phone_watch[df_phone_watch['activity']=='I'].head(174604)
df_j = df_phone_watch[df_phone_watch['activity']=='J']

In [24]:
balanced_data = pd.DataFrame()
balanced_data = balanced_data.append([df_a,df_m,df_k,df_p,df_e,df_o,df_c,df_d,df_l,df_b,df_h,df_f,df_g,df_q,df_r,df_s,df_i,df_j]) 

In [25]:
balanced_data['activity'].value_counts()

J    180895
R    174604
B    174604
M    174604
C    174604
K    174604
P    174604
L    174604
F    174604
Q    174604
I    174604
E    174604
G    174604
S    174604
D    174604
H    174604
A    174604
O    174604
Name: activity, dtype: int64

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
label = LabelEncoder()
balanced_data['label'] = label.fit_transform(balanced_data['activity']) 
balanced_data

,activity,x,y,z,label
0,A,13.750549,-11.874284,-7.126030,0
1,A,8.361988,-18.150660,-4.347719,0
2,A,6.586178,-11.288211,-5.160292,0
3,A,10.606791,-13.405935,-4.386877,0
4,A,13.392424,-12.947109,-7.125218,0
...,...,...,...,...,...
45494,J,-2.656428,-6.847281,-0.794544,9
45495,J,19.716516,-7.460141,14.250988,9
45496,J,-10.976937,-10.508259,11.279987,9
45497,J,-10.936750,-12.066632,-3.021969,9


In [28]:
label.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'O', 'P', 'Q', 'R', 'S'], dtype=object)

**Statndardize Data**

In [29]:
from sklearn.preprocessing import StandardScaler

x = balanced_data[['x','y','z']]
y = balanced_data['label']
scaler = StandardScaler()
x = scaler.fit_transform(x)

scaled_x = pd.DataFrame(data=x, columns=['x','y','z'])
scaled_x['label'] = y.values

scaled_x

,x,y,z,label
0,1.485231,-0.661030,-1.194153,0
1,0.888257,-1.370361,-0.829636,0
2,0.691523,-0.594794,-0.936247,0
3,1.136949,-0.834131,-0.834774,0
4,1.445556,-0.782276,-1.194046,0
...,...,...,...,...
3149158,-0.332424,-0.092898,-0.363456,9
3149159,2.146174,-0.162161,1.610531,9
3149160,-1.254216,-0.506647,1.220733,9
3149161,-1.249764,-0.682768,-0.655697,9


**Frame Preparation**

In [30]:
import scipy.stats as stats

In [31]:
Fs=20
frame_size = Fs*4 #80
hop_size = Fs*2 #40

In [32]:
def get_frames(df, frame_size, hop_size):
    
    N_FEATURES = 3
    frames = []
    labels = []
    for i in range(0,len(df )- frame_size, hop_size):
        x = df['x'].values[i: i+frame_size]
        y = df['y'].values[i: i+frame_size]
        z = df['z'].values[i: i+frame_size]
        
        label = stats.mode(df['label'][i: i+frame_size])[0][0]
        frames.append([x,y,z])
        labels.append(label)
        
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [33]:
x,y = get_frames(scaled_x, frame_size, hop_size)

In [34]:
 x.shape, y.shape

((78728, 80, 3), (78728,))

In [35]:
from sklearn.model_selection import train_test_split

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30, random_state = 0, stratify = y)

In [58]:
y_train.shape

(55109,)

In [70]:
x_train.shape, x_test.shape

((55109, 80, 3), (23619, 80, 3))

In [72]:
x_train[0].shape, x_test[0].shape

((80, 3), (80, 3))

In [74]:
x_train = x_train.reshape(55109, 80, 3,1)
x_test = x_test.reshape(23619, 80, 3,1)

**2D CNN Model**

In [78]:
model = Sequential()
model.add(Conv2D(256, (2,2), activation = 'relu', input_shape = x_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(512, (2,2), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(18, activation='softmax'))

In [79]:
model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]) 

In [80]:
history = model.fit(x_train, y_train, epochs = 25, validation_data=(x_test, y_test), verbose=1 )

Train on 55109 samples, validate on 23619 samples
Epoch 1/25
55109/55109 [==============================] - 35s 634us/step - loss: 2.3569 - accuracy: 0.2395 - val_loss: 1.9909 - val_accuracy: 0.3701
Epoch 2/25
55109/55109 [==============================] - 35s 628us/step - loss: 1.8530 - accuracy: 0.4017 - val_loss: 1.6498 - val_accuracy: 0.4743
Epoch 3/25
55109/55109 [==============================] - 35s 627us/step - loss: 1.5074 - accuracy: 0.5152 - val_loss: 1.4385 - val_accuracy: 0.5451
Epoch 4/25
55109/55109 [==============================] - 35s 633us/step - loss: 1.2545 - accuracy: 0.5943 - val_loss: 1.3385 - val_accuracy: 0.5761
Epoch 5/25
55109/55109 [==============================] - 34s 621us/step - loss: 1.0758 - accuracy: 0.6508 - val_loss: 1.3389 - val_accuracy: 0.5925
Epoch 6/25
55109/55109 [==============================] - 35s 627us/step - loss: 0.9457 - accuracy: 0.6901 - val_loss: 1.3119 - val_accuracy: 0.6014
Epoch 7/25
55109/55109 [==============================] 